# Overview



# Paths and parameters
You'll need to execute the next cell for all cells that follow.

In [ ]:
import sys
import os
import def_definepaths as dd
import def_acquisition as da
import numpy as np
import cv2 as cv

# Get paths (specific to system running code)
path = dd.give_paths()

# Add path to kineKit 'sources' directory
sys.path.insert(0, path['kinekit'] + os.sep + 'sources')

# Import from kineKit
import acqfunctions as af
import videotools as vt

# Raw video extension
vid_ext_raw = 'MOV'

# Video quality level (0 to 1, where 1 is no compression)
vid_quality = 0.75

# Create mask

You'll need to create a new mask whenever the position of the tank changes in the video frame. Follow the following steps to create a new mask.

## Measure the region-of-interest (ROI)

Here the aim is to define an ellipical region-of-interest where the tank resides within the video frames.

1. Set the 'make_video' column in the 'experiment_log.csv' file to a value of 1 for the video for which you want to create a mask.

1. Set all other 'make_video' rows to 0. If there is more than one row set to 1 then the code will use the first in the list.

1. Run the cell below to save a single frame from the video, which is saved to the 'masks' directory.

1. After the frame is generated, open it up in ImageJ, use the ellipse tool to draw around the tank margin. Be sure not to cut off any portion of the tank where a fish might end up. All areas outside of the ROI will be ignored.

1. Once the ellipse has been drawn, select Analyze:Measure in the pull-down menus to find the region-of-interest coordinates.

1. Enter the value for 'BX' as 'roi_x' in the experiment_log (enter values online in Google Sheets). Do the same for BY->roi_y, Width->roi_w, Height->roi_h. Copy and paste the values to all rows corresponding to videos that use that same ROI.

1. Make sure that the local version of experiment_log matches the values as the Google Sheet. This can be done by redownloading the CSV file, or copy and pasting values to the local copy. 



In [ ]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Index of video in cat list to extract video
vid_index = 0

# Define path
full_path = path['vidin'] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

# Extract frame and save to 'mask' directory
im = vt.get_frame(full_path)
cv.imwrite(path['mask'] + os.sep + 'frame_from_' + cat.video_filename[vid_index] + '.jpg', im)

## Create the mask image

1. As in step above, the cell below will use the first row for which make_video=1 in the 'experiment_log.csv' file to define the ROI for the mask, so adjust the spreadsheet accordingly.

1. The code will prompt you to choose a filename for the mask image and will save that file to the 'masks' directory.

1. Once completed, enter the mask filename (without the 'png' extension) into the mask_filename column of experiment_log for all experiments that should use that mask.

In [ ]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Index of video in cat list to extract video
vid_index = 0

# Define path
full_path = path['vidin'] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

# Extract video frame 
im = vt.get_frame(full_path)

# Extract roi coordinates
x_roi = float(cat.roi_x[vid_index])
y_roi = float(cat.roi_y[vid_index])
w_roi = float(cat.roi_w[vid_index])
h_roi = float(cat.roi_h[vid_index])
xC = x_roi + w_roi/2
yC = y_roi + h_roi/2
dims = (int(np.ceil(w_roi/2)), int(np.ceil(h_roi/2)))
cntr = (int(x_roi + w_roi/2), int(y_roi + h_roi/2))

# Define transparent image for mask
im = cv.ellipse(im, cntr, dims, angle=0, startAngle=0, endAngle=360, color=(255,255,255),thickness=-1)
trans_img = int(255/3) * np.ones((im.shape[0], im.shape[1], 4), dtype=np.uint8)
trans_img[np.where(np.all(im[..., :3] == 255, -1))] = 0

# Write mask file to disk
while True:
    filename = input('What filename do you want to use for the mask?')
    if os.path.isfile(path['mask'] + os.sep + filename + '.png'):
        print('The filename ' + filename + ' already exists. Try again.')
    else:
        cv.imwrite(path['mask'] + os.sep + filename + '.png', trans_img)
        print('Mask file saved: ' + path['mask'] + os.sep + filename + '.png')
        break

# Create compressed movies

The code below will generate compressed videos for all experiments in experiment_log where analyze=1 and make_video=1. 
This is done in two steps.
First, uncompressed masked movies are created and stored in the 'tmp' directory, then compressed and cropped movies are saved in 'pilot_compressed' (the tmp movies are then deleted).

In [ ]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Make the masked videos (stored in 'tmp' directory)
print(' ')
print('=====================================================')
print('First, creating masked videos . . .')
af.convert_masked_videos(cat, in_path=path['vidin'], out_path=path['tmp'], maskpath=path['mask'], vmode=False, imquality=1)


# Make the downsampled/cropped videos  (stored in 'pilot_compressed' directory)
print(' ')
print('=====================================================')
print('Second, creating downsampled and cropped videos . . .')
af.convert_videos(cat, in_path=path['tmp'], out_path=path['vidout'], vmode=False, imquality=vid_quality, suffix_in='mp4')

# Survey resulting directories 
# Loop thru each video listed in cat
print(' ')
print('=====================================================')
print('Surveying results . . .')
for c_row in cat.index:
    # Input video path
    vid_in_path = path['vidin'] + os.sep + cat.video_filename[c_row] + '.' + os.sep + vid_ext_raw

    # Temp video path
    vid_tmp_path = path['tmp'] + os.sep + cat.video_filename[c_row] + '.mp4'

    # Output video path
    vid_out_path = path['vidout'] + os.sep + cat.video_filename[c_row] + '.mp4'

    # Check that output file was made
    if not os.path.isfile(vid_out_path):

        print('   Output movie NOT created successfully: ' + vid_out_path)

        if os.path.isfile(vid_tmp_path):
            print('   Also, temp. movie NOT created successfully: ' + vid_tmp_path)
        else:
            print('   But, temp. movie created successfully: ' + vid_tmp_path)
    else:

        print('   Output movie created successfully: ' + vid_out_path)

        # Delete temp file
        if os.path.isfile(vid_tmp_path):
            os.remove(vid_tmp_path)